In [3]:
import os
os.environ["OMP_NUM_THREADS"]='1'
import osmnx as ox
import networkx as nx
from datetime import datetime

import pandas as pd
import geopandas as gpd

from pyproj import Transformer 
import numpy as np
import scipy
from descartes import PolygonPatch

import matplotlib.pyplot as plt
from matplotlib import *
from matplotlib.patches import *
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors

import contextily as cx #Copyright (c) 2016, Dani Arribas-Bel. All rights reserved.

import math
import itertools
from shapely.geometry import MultiPolygon, Polygon, Point, LineString
from shapely.ops import nearest_points
from IPython.display import Image, IFrame

from sklearn import cluster
from sklearn.cluster import KMeans


import pandapower as pp
import pandapower.timeseries as ts
import pandapower.control as control
from pandapower.plotting import simple_plot, simple_plotly, pf_res_plotly 
import pandapower.plotting.plotly as plotly


In [4]:
def delete_all_components(net):
    # Alle Busse löschen
    net.bus.drop(net.bus.index, inplace=True)
    
    # Alle Erzeugungen löschen
    net.gen.drop(net.gen.index, inplace=True)
    net.sgen.drop(net.sgen.index, inplace=True)
    net.storage.drop(net.storage.index, inplace=True)

    # Alle Lasten löschen
    net.load.drop(net.load.index, inplace=True)
    
    # Alle Leitungen löschen
    net.line.drop(net.line.index, inplace=True)
    
    # Alle externen Netze löschen
    net.ext_grid.drop(net.ext_grid.index, inplace=True)

In [5]:
def initialize_network():
    # Erstellen eines neuen leeren Netzwerks
    return pp.from_json('Daten/Netzmodell_PandaPower_Weilimdorf.json')

In [6]:
def extract_results(net):
    # Maximale Belastung der Kabel (in %)
    max_line_loading = net.res_line.loading_percent.max()
    
    # Maximale Belastung der Trafos (in %)
    if len(net.trafo) > 0:
        max_trafo_loading = net.res_trafo.loading_percent.max()
    else:
        max_trafo_loading = 0  # Falls keine Trafos im Netz sind

    # Gesamte Erzeugungsleistung (in MW)
    total_generation = net.res_gen.p_mw.sum() + net.res_sgen.p_mw.sum() + net.res_storage.p_mw.sum()
    
    # Gesamte Last (in MW)
    total_load = net.res_load.p_mw.sum()
    
    # Weitere interessante Parameter
    max_bus_voltage = net.res_bus.vm_pu.max()
    min_bus_voltage = net.res_bus.vm_pu.min()
    total_losses = net.res_line.pl_mw.sum() + net.res_trafo.pl_mw.sum()

    return {
        "max_line_loading": max_line_loading,
        "max_trafo_loading": max_trafo_loading,
        "total_generation": total_generation,
        "total_load": total_load,
        "max_bus_voltage": max_bus_voltage,
        "min_bus_voltage": min_bus_voltage,
        "total_losses": total_losses
    }

In [7]:
def find_rings(net):
    visited_lines = set()
    rings = []

    for line_idx in net.line.index:
        if line_idx in visited_lines:
            continue
        
        # Fange einen neuen Ring an
        stack = [line_idx]
        ring_lines = set()
        ring_buses = set()

        while stack:
            current_line = stack.pop()
            
            if current_line in visited_lines:
                continue
            
            visited_lines.add(current_line)
            ring_lines.add(current_line)
            
            # Füge die Busse der aktuellen Leitung hinzu
            from_bus = net.line.at[current_line, 'from_bus']
            to_bus = net.line.at[current_line, 'to_bus']
            ring_buses.update([from_bus, to_bus])

            # Finde benachbarte Leitungen (die zu den aktuellen Bussen führen)
            connected_lines = net.line[((net.line['from_bus'] == from_bus) | 
                                        (net.line['to_bus'] == from_bus) |
                                        (net.line['from_bus'] == to_bus) | 
                                        (net.line['to_bus'] == to_bus))].index
            
            # Füge alle benachbarten Leitungen, die noch nicht besucht wurden, zum Stack hinzu
            for next_line in connected_lines:
                if next_line not in visited_lines:
                    stack.append(next_line)
        
        if ring_lines:
            rings.append((ring_buses, ring_lines))

    return rings

In [8]:
def export_network_parameters(net):
    # Erstelle eine Liste, um die Parameter zu speichern
    data = []

    # Gesamtanzahl der Busse
    total_buses = len(net.bus)
    data.append(['total_buses', total_buses])
    
    # Anzahl der Verbraucher
    total_loads = len(net.load)
    data.append(['total_loads', total_loads])

    # Anzahl der Transformatoren (Ortsnetzstationen)
    total_transformers = len(net.trafo)
    data.append(['total_transformers', total_transformers])
    
    # Gesamtanzahl der Leitungen
    total_lines = len(net.line)
    data.append(['Anzahl Leitungen', total_lines])

    # Gesamtlänge aller Leitungen
    total_line_length_km = net.line['length_km'].sum()
    data.append(['Gesamtlänge Leitungen (km)', total_line_length_km])
    
    # Finde alle Ringe im Netz
    rings = find_rings(net)

    # Anzahl der erkannten Ringe
    total_rings = len(rings)
    data.append(['Anzahl Ringe', total_rings])
    data.append(['Durchschnittliche Leitungslänge pro Ring', total_line_length_km/total_rings])

    # Optional: Details über jeden Ring
    for i, (buses, lines) in enumerate(rings):
        data.append([f'Leitungslänge Ring {i} (km)', net.line.loc[list(lines), 'length_km'].sum()])
    
    # Konvertiere die Liste in ein DataFrame
    df_parameters = pd.DataFrame(data, columns=['Parameter', 'Value'])
    
    return df_parameters

In [9]:
net = initialize_network()
param = export_network_parameters(net)
delete_all_components(net)
param

,Parameter,Value
0,total_buses,256.00000
1,total_loads,50.00000
2,total_transformers,2.00000
3,Anzahl Leitungen,166.00000
4,Gesamtlänge Leitungen (km),15.69500
5,Anzahl Ringe,4.00000
6,Durchschnittliche Leitungslänge pro Ring,3.92375
7,Leitungslänge Ring 0 (km),2.87800
8,Leitungslänge Ring 1 (km),2.08000
9,Leitungslänge Ring 2 (km),8.57600


In [10]:

def lade_messdaten(net, P):
    #Wird nur einmal benötigt

    # Erstelle einen leeren DataFrame, der später gefüllt wird
    df = pd.DataFrame()
    
    # Durchlaufe alle Busse im Netzwerk
    for bus in net.bus.name:
        # Konvertiere den Bus-Index in einen String, falls es sich um eine Zahl handelt
        bus_str = str(bus)

        # Extrahiere die Nummer des Busses aus dem Namen (z.B. "AbA 3300" -> "3300")
        bus_nummer = ''.join(filter(str.isdigit, bus_str))
        if len(bus_nummer) != 4:
            continue
        # Suche nach dem Ordner, der zu dieser Bus-Nummer gehört
        messordner = None
        for ordner in os.listdir('./Messungen'):
            ordnerpfad = os.path.join('./Messungen', ordner)

            if os.path.isdir(ordnerpfad) and bus_nummer in ordner:  # Überprüfen, ob es sich um einen Ordner handelt
                messordner = ordnerpfad
                break

        # Falls kein passender Ordner gefunden wurde, überspringe diesen Bus
        if not messordner:
            continue
        
        # Durchsuche den gefundenen Ordner nach der Datei, die mit der Busnummer beginnt
        for datei in os.listdir(messordner):
            if datei.startswith(bus_nummer) and datei.endswith(P + '.xlsx'):
                # Lade die Excel-Datei
                dateipfad = os.path.join(messordner, datei)
                df_temp = pd.read_excel(dateipfad, usecols=[2], header=0, names=[bus])
                
                # Füge die Daten dem DataFrame hinzu
                if df.empty:
                    df = df_temp
                else:
                    df = pd.concat([df, df_temp], axis=1)
                break
    
    return df

In [11]:
def lade_messdaten_normal(net,type):
    if type == "P+":
        return pd.read_excel('./Messungen/p_messungen_1.xlsx')
    elif type == "Q1":
        return pd.read_excel('./Messungen/q_messungen_1.xlsx')
    elif type == "P-":
        return pd.read_excel('./Messungen/p_gen_messungen_1.xlsx')
    else:
        print("Fehler")

In [12]:
#net = initialize_network()

result_df_p = lade_messdaten_normal(net,"P+")
result_df_q = lade_messdaten_normal(net,"Q1")
result_df_p_gen = lade_messdaten_normal(net,"P-")
#result_df_p.to_excel('./Messungen/p_messungen_1.xlsx')
#result_df_q.to_excel('./Messungen/q_messungen_1.xlsx')
#result_df_p_gen.to_excel('./Messungen/p_gen_messungen_1.xlsx')

#delete_all_components(net)

In [13]:
messdaten_p_Winter = result_df_p[1537:2209]
messdaten_p_gen_Winter = result_df_p_gen[1537:2209]
messdaten_q_Winter = result_df_q[1537:2209]

messdaten_p_Sommer = result_df_p[22177:22849]
messdaten_p_gen_Sommer = result_df_p_gen[22177:22849]
messdaten_q_Sommer = result_df_q[22177:22849]

messdaten_p_Fruehling = result_df_p[10177:10849]
messdaten_p_gen_Fruehling = result_df_p_gen[10177:10849]
messdaten_q_Fruehling = result_df_q[10177:10849]


In [49]:
def update_loads(net, load_p, load_q, generation):
    # Alle Lasten löschen
    net.load.drop(net.load.index, inplace=True)
    net.sgen.drop(net.sgen.index, inplace=True)

    if isinstance(load_p, pd.Series):
        load_df_p = load_p.to_frame()
    else:
        load_df_p = load_p

    if isinstance(load_q, pd.Series):
        load_df_q = load_q.to_frame()
    else:
        load_df_q = load_q

    if isinstance(generation, pd.Series):
        gen_df = generation.to_frame()
    else:
        gen_df = generation


    for bus in net.bus.name:
        bus_name = str(bus)
        Flag = 0
        if bus_name in load_df_p.index:
            try:
                # Werte für P und Q holen
                P = load_df_p.loc[bus_name].iloc[0]
                if bus_name in load_df_q.index:
                    Q = load_df_q.loc[bus_name].iloc[0] 
                else:
                    Q=0

                if bus_name in gen_df.index:
                    gen = gen_df.loc[bus_name].iloc[0]
                    pp.create_sgen(net,bus_idx,p_mw = gen)


                # Den Bus-Index im Pandapower-Netzwerk finden
                bus_idx = net.bus.index[net.bus['name'] == bus_name].tolist()
                if bus_idx:
                    bus_idx = bus_idx[0]  # Den ersten (und wahrscheinlich einzigen) Bus-Index verwenden
                    pp.create_load(net, bus_idx, p_mw=P, q_mvar=Q)
              
            except KeyError:
                print(f"Spalte für '{bus_name}' nicht gefunden.")

    

In [50]:
results = []
plot = False
P_Zeitreihe = messdaten_p_Winter
Q_Zeitreihe = messdaten_q_Winter
Gen_Zeitreihe = messdaten_p_gen_Winter


for j in range(len(P_Zeitreihe)):
    p = P_Zeitreihe.iloc[j]/1000
    q = Q_Zeitreihe.iloc[j]/1000
    gen = Gen_Zeitreihe.iloc[j]/1000

    if j%10 == 0:
        print(f"Iteration {j}/{len(P_Zeitreihe)}")
    
    # Netzwerk initialisieren
    if j == 0:
        net1 = initialize_network()
    
    update_loads(net1,p,q,gen)
    
    # Power Flow Berechnung durchführen
    pp.runpp(net1, algorithm='nr', error_tolerance_vm_pu=1e-08, symmetric=True, validate_input=False)

    # Ergebnisse speichern 
    result_data = extract_results(net1)
    result_data.update({
        "Zeitschritt": j,
    })
    results.append(result_data)

    if j%48 == 0 and plot:
        pp.plotting.pf_res_plotly(net1)    
    
    #Ergebnisse speichern
    if j == len(P_Zeitreihe) - 1:
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_name_net = f"./results/Weilimdorf_{current_time}_pp_net.xlsx"
        file_name_result = f"./results/Weilimdorf_{current_time}_pp_net_result.xlsx"
        df = pd.DataFrame(results)
        pp.to_excel(net1, file_name_net)
        df.to_excel(file_name_result)
        break  # Abbrechen der Schleife vor der letzten Iteration
    

Iteration 0/672
Iteration 10/672
Iteration 20/672
Iteration 30/672
Iteration 40/672
Iteration 50/672
Iteration 60/672
Iteration 70/672
Iteration 80/672
Iteration 90/672
Iteration 100/672
Iteration 110/672
Iteration 120/672
Iteration 130/672
Iteration 140/672
Iteration 150/672
Iteration 160/672
Iteration 170/672
Iteration 180/672
Iteration 190/672
Iteration 200/672
Iteration 210/672
Iteration 220/672
Iteration 230/672
Iteration 240/672
Iteration 250/672
Iteration 260/672
Iteration 270/672
Iteration 280/672
Iteration 290/672
Iteration 300/672
Iteration 310/672
Iteration 320/672
Iteration 330/672
Iteration 340/672
Iteration 350/672
Iteration 360/672
Iteration 370/672
Iteration 380/672
Iteration 390/672
Iteration 400/672
Iteration 410/672
Iteration 420/672
Iteration 430/672
Iteration 440/672
Iteration 450/672
Iteration 460/672
Iteration 470/672
Iteration 480/672
Iteration 490/672
Iteration 500/672
Iteration 510/672
Iteration 520/672
Iteration 530/672
Iteration 540/672
Iteration 550/672
Ite